<div align="center">,
    <img src = "./assets/Nebari-Logo-Horizontal-Lockup.svg" alt="Dask logo" width="25%">
</div>

---

# How to setup and deploy your own Nebari cluster

## 💎 Key takeaways

By the end of this notebook, you will:
- **Know how to use the `nebari` CLI to initialize and deploy a Nebari cluster of your own,**
- **Get an introduction to managing your Nebari cluster via GitOps and**
- **Get an introduction to Keycloak for managing users and groups on your Nebari cluster.**

It's worth pausing briefly to highlight that all of the features and integrations demoed here today come pre-installed and pre-configured for you. 

There is nothing to stopping you from deploying your own Zero-to-JupyterHub and integrating these features and services yourself. That said, the Nebari development team has spent hundreds of hours making all of these particulars changes while also making the deployment and management process as simple and as straight-forward as possible. 

Nebari, as a data-science platform, is being used in production and at-scale by organizations across multiple domains including finance, geospatial and advertising. 

If you choose to use Nebari, you're in good hands 😊

## Getting started with Nebari

Up until now, the focus of this tutorial has been from the perspective of a user, such as a data scientist or machine learning engineer. Features a like conda-store and Dask Gateway make it easy for them to quickly jump into their work without having to concern themselves with cluster management and adminstration.

But how do we deploy a Nebari cluster?

One of the major benefits of using Nebari is the relative ease with which it can be deployed. 

> Nebari has a DevOps for non-DevOps people approach 🥳

Nebari comes with a command-line interface (CLI) - `nebari` - that is used to: 
1. generate and initialize your `nebari-config.yaml`
2. deploy Nebari on the cloud of your choose
3. destroy Nebari and all resources (if and when the time comes)

<img src="./assets/nebari-cli-commands.png" alt="Diagram of the Nebari CLI commands" width="50%"></img>

If you can generate a handful of cloud secrets and API keys, you can deploy Nebari!

To get started, visit the [Nebari Quickstart](https://www.nebari.dev/docs/get-started/quickstart) guide.

## Step 1: generate and initialize your `nebari-config.yaml`

The `nebari init` command is used once, during your initial setup, to generate and initialize your `nebari-config.yaml`. This configuration file is used to define the specifics your particular Nebari cluster, this includes items such:
- cloud provider that you wish to deploy to
    - [GCP](https://www.nebari.dev/docs/how-tos/nebari-gcp), [AWS](https://www.nebari.dev/docs/how-tos/nebari-aws/), [Digital Ocean](https://www.nebari.dev/docs/how-tos/nebari-do/), and [Azure](https://www.nebari.dev/docs/how-tos/nebari-azure) or on any existing Kubernetes cluster on other major cloud providers.
- domain name you wish to access your cluster from
- identity provider you wish to integrate with (GitHub, Auth0, etc.)
- and much more.

You can also integrate other Helm-based apps not included in Nebari by adding them to the `helm_extensions` section of the `nebari-config.yaml`.

### `nebari init --guided-init`

The `nebari init` command has many available options and can be a little intimidating to use at first. To make it easier on folks who are coming to the project, we added a `nebari init --guided-init` command which walks users through a set of questions to generate the `nebari-config.yaml`.

### An example `nebari-config.yaml`

<details>
    <summary>An example `nebari-config.yaml`</summary>
    
```yaml

nebari_version: 2022.10.1
project_name: demo-nebari
provider: gcp
domain: demo.nebari.dev
namespace: dev
terraform_state:
  type: remote
certificate:
  type: lets-encrypt
  acme_email: internal-it@quansight.com
  acme_server: https://acme-v02.api.letsencrypt.org/directory
security:
  authentication:
    type: Auth0
    config:
      client_id: <redacted>
      client_secret: <redacted>
      auth0_subdomain: nebari-dev
  keycloak:
    initial_root_password: <redacted>
default_images:
  jupyterhub: quay.io/nebari/nebari-jupyterhub:main
  jupyterlab: quay.io/nebari/nebari-jupyterlab:main
  dask_worker: quay.io/nebari/nebari-dask-worker:main
storage:
  conda_store: 200Gi
  shared_filesystem: 200Gi
theme:
  jupyterhub:
    hub_title: demo.nebari.dev
    hub_subtitle: Your open source data science platform, hosted on Google Cloud Platform
    welcome: Welcome! Learn about Nebari's features and configurations in <a href="https://www.nebari.dev/docs">the
      documentation</a>. If you have any questions or feedback, reach the team on
      <a href="https://www.nebari.dev/docs/community#getting-support">Nebari's support
      forums</a>.
    logo: https://raw.githubusercontent.com/nebari-dev/nebari-design/main/logo-mark/horizontal/Nebari-Logo-Horizontal-Lockup-White-text.svg
    display_version: true
helm_extensions: []
monitoring:
  enabled: true
argo_workflows:
  enabled: true
kbatch:
  enabled: true
cdsdashboards:
  enabled: true
  cds_hide_user_named_servers: true
  cds_hide_user_dashboard_servers: false
ci_cd:
  type: github-actions
  branch: main
  commit_render: true
google_cloud_platform:
  project: qhub-279316
  region: us-central1
  kubernetes_version: 1.23.11-gke.300
  node_groups:
    general:
      instance: n1-standard-8
      min_nodes: 1
      max_nodes: 1
    user:
      instance: n1-standard-4
      min_nodes: 0
      max_nodes: 5
    worker:
      instance: n1-standard-4
      min_nodes: 0
      max_nodes: 5
    gpu-tesla-t4:
      instance: "n1-standard-8"
      min_nodes: 0
      max_nodes: 2
      guest_accelerators:
        - name: nvidia-tesla-t4    # 16 GB GDDR6: Nividia Tesla T4
          count: 1
profiles:
  jupyterlab:
  - display_name: Small Instance
    description: Stable environment with 2 cpu / 8 GB ram
    default: true
    kubespawner_override:
      cpu_limit: 2
      cpu_guarantee: 1.5
      mem_limit: 8G
      mem_guarantee: 5G
  - display_name: Medium Instance
    description: Stable environment with 4 cpu / 16 GB ram
    kubespawner_override:
      cpu_limit: 4
      cpu_guarantee: 3
      mem_limit: 16G
      mem_guarantee: 10G
  - display_name: Large GPU Instance
    description: Stable environment with 8 cpu / 32 GB ram and 1 Nvidia Tesla T4
    kubespawner_override:
      image: quay.io/nebari/nebari-jupyterlab-gpu:main
      cpu_limit: 8
      cpu_guarantee: 7.25
      mem_limit: 32G
      mem_guarantee: 24G
      extra_resource_limits:
        nvidia.com/gpu: 1
      node_selector:
          "cloud.google.com/gke-nodepool": "gpu-tesla-t4"
  dask_worker:
    Small Worker:
      worker_cores_limit: 2
      worker_cores: 1.5
      worker_memory_limit: 8G
      worker_memory: 5G
      worker_threads: 2
    Medium Worker:
      worker_cores_limit: 4
      worker_cores: 3
      worker_memory_limit: 16G
      worker_memory: 10G
      worker_threads: 4
environments:
  environment-dask.yaml:
    name: dask
    channels:
    - conda-forge
    dependencies:
    - python
    - ipykernel
    - ipywidgets==7.7.1
    - nebari-dask ==2022.10.1
    - python-graphviz
    - pyarrow
    - s3fs
    - gcsfs
    - numpy
    - numba
    - pandas
    - pip:
      - kbatch
  environment-dashboard.yaml:
    name: dashboard
    channels:
    - conda-forge
    dependencies:
    - python==3.9.13
    - ipykernel==6.15.1
    - ipywidgets==7.7.1
    - nebari-dask ==2022.10.1
    - param==1.12.2
    - python-graphviz==0.20.1
    - matplotlib==3.3.2
    - panel==0.13.1
    - voila==0.3.6
    - streamlit==1.10.0
    - dash==2.6.1
    - cdsdashboards-singleuser==0.6.2
```
   
</details>

## Step 2: deploy Nebari

Once generated, this `nebari-config.yaml` you can be manually edit it to make any additional updates to the cluster. Then when you're ready you, and with you cloud credentials set, simply run the following command to deploy your Nebari cluster.

```
nebari deploy -c nebari-config.yaml
```

This command will validate your config, render the Terraform scripts and then deploys your cluster. It will create all the cloud resources you need and install all of the features and services we just covered. 

> The initial deployment takes anywhere from 25 minutes to an hour (depending on cloud provider and DNS domain propagation). 

> Future deployments will only take a fraction of the time with no interruption to the platform users. 

That's it. From here you can upload your infrastructure code to a shared repository and manage your cluster via a GitOps approach (see below for more details).

## 👀 Watch this:

*Show Nebari CLI (guided-init, render, validate, deploy) and basic deployment from the terminal.*

## Managing your Nebari cluster via [GitOps](https://www.gitops.tech/) 

For those unfamiliar, GitOps "is a way of implementing Continuous Deployment for cloud native applications" (source: [gitops.tech](https://www.gitops.tech/)). This means your deployment is handled by a CI/CD platform such as GitHub-Actions or GitLab CI and that your infrastructure code (i.e. `nebari-config.yaml`, and the Terraform scripts) live in a shared repository that you can share with your team.

Without using a GitOps approach, you make the appropriate change to the `nebari-config.yaml` and then redeploy from your local machine. For this to work, you will need to ensure you have `nebari` installed, your cloud credentials set and that you have a stable internet connetion. This puts responsibility of the deployment in the hands of one person.

With a GitOps approach, you make the same change to the `nebari-config.yaml` but now, you open a pull-request against the repo hosting this infrastructure code. From here, team members can chime in and approve the change. Once everyone is happy, you merge your changes which then triggers a redeployment of your Nebari cluster using GitHub-Actions (or GitLab CI).

> Your cloud credentials need to be added as repository secrets.

## Identity and access management (IAM) with [Keycloak](https://www.keycloak.org/)

Throughout the tutorial, we have made passing references to Keycloak and the ability to grant or restrict access to particular resources and platform features. With Keycloak, administrators can ensure each user has the appropriate access to the resources they need to perform their job duties. This is the idea by the concept of ["least privilege access"](https://en.wikipedia.org/wiki/Principle_of_least_privilege).

For example, not all users need the ability to launch Dask Gateway clusters. With Keycloak, you can restrict access to Dask Gateway to only those users who need this feature to perform their job.

For more information on how to configure Keycloak on your Nebari cluster, visit our [Configure Keycloak](https://nebari-docs.netlify.app/docs/how-tos/configuring-keycloak) docs.

## 👀 Watch this:

*Show Keycloak admin console.*

---
## 👏 Next:
* [06_conclusion](./06_conclusion.ipynb)
---